<a href="https://colab.research.google.com/github/RithvikManda/GenAI/blob/main/RAG_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain openai tiktoken rapidocr-onnxruntime langchain-community langchain_groq faiss-cpu pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.2 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
GROQ_API_KEY=userdata.get('GROQ_API_KEY')

In [3]:
import os
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [6]:
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS

In [77]:
!pip install --upgrade pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 79.4 MB/s eta 0:00:00


In [79]:
import fitz
with fitz.open("/content/3-90's.pdf") as pdf:
    data = ""
    for page in pdf:
        data += page.get_text()

print(data)


DSA 
 
WEEK 1: Problems on Mathematics, Bit manipulation 
MATHEMATICS 
1. Prime Numbers: Write a program to check if a given number is prime. Extend it to 
print all prime numbers up to n. 
2. GCD and LCM: Implement a function to calculate the GCD (Greatest Common 
Divisor) and LCM (Least Common Multiple) of two numbers using the Euclidean 
algorithm. 
3. Sieve of Eratosthenes: Generate all prime numbers up to n using the Sieve of 
Eratosthenes method. 
4. Modular Arithmetic: Solve abmod  ca^b \mod c (modular exponentiation) efficiently 
using recursion or iteration. 
5. Trailing Zeros in Factorial: Write a program to find the number of trailing zeros in 
n!. 
6. Perfect Numbers: Determine whether a given number is a perfect number (a number 
equal to the sum of its proper divisors). 
7. Fibonacci Numbers: Find the n-th Fibonacci number using recursion, dynamic 
programming, and matrix exponentiation. 
8. Number of Divisors: Write a program to calculate the number of divisors for a giv

In [81]:
with open("dsa.txt", "w") as file:
    file.write(data)

In [82]:
loder=TextLoader('dsa.txt', encoding="utf8")

In [83]:
document=loder.load()

In [84]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=400,chunk_overlap=50)
text_chunks=text_splitter.split_documents(document)

In [85]:
print(text_chunks)

[Document(metadata={'source': 'dsa.txt'}, page_content='DSA \n \nWEEK 1: Problems on Mathematics, Bit manipulation \nMATHEMATICS \n1. Prime Numbers: Write a program to check if a given number is prime. Extend it to \nprint all prime numbers up to n. \n2. GCD and LCM: Implement a function to calculate the GCD (Greatest Common \nDivisor) and LCM (Least Common Multiple) of two numbers using the Euclidean \nalgorithm.'), Document(metadata={'source': 'dsa.txt'}, page_content='algorithm. \n3. Sieve of Eratosthenes: Generate all prime numbers up to n using the Sieve of \nEratosthenes method. \n4. Modular Arithmetic: Solve abmod\u2009\u2009ca^b \\mod c (modular exponentiation) efficiently \nusing recursion or iteration. \n5. Trailing Zeros in Factorial: Write a program to find the number of trailing zeros in \nn!.'), Document(metadata={'source': 'dsa.txt'}, page_content='n!. \n6. Perfect Numbers: Determine whether a given number is a perfect number (a number \nequal to the sum of its proper di

In [86]:
print(text_chunks[3].page_content)

number n. 
9. Armstrong Numbers: Check if a given number is an Armstrong number (e.g., 153 is 
an Armstrong number because 13+53+33=1531^3 + 5^3 + 3^3 = 153). 
10. Count Digits: Write a function to count the number of digits in a given number and 
determine if it is a palindrome. 
BIT MANIPULATION 
1. Check if a Number is Odd or Even: Write a program to determine if a number is


In [87]:
from transformers import AutoModel
from langchain.vectorstores import FAISS
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

In [88]:
vectorstore = FAISS.from_documents(text_chunks, embeddings)

In [89]:
retriever=vectorstore.as_retriever()

In [90]:
from langchain.prompts import ChatPromptTemplate

In [91]:
template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know this is important don't provide the answers which are not present in the context.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [92]:
prompt=ChatPromptTemplate.from_template(template)

In [93]:
from langchain_groq import ChatGroq
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [94]:
output_parser=StrOutputParser()

In [95]:
llm_model = ChatGroq(
    temperature=0,
    groq_api_key=GROQ_API_KEY,
    model_name="mixtral-8x7b-32768",
)

In [96]:
rag_chain = (
    retriever  # The retriever takes only the query (question) as input
    | (lambda docs: {"context": docs, "question": RunnablePassthrough()})  # Combine context with the question
    | prompt
    | llm_model
    | StrOutputParser()
)

In [98]:
result = rag_chain.invoke("Give me 5 DSA questions on Tree")
print(result)

The context provided includes several programming tasks related to binary trees, including inorder traversal of a binary search tree, finding the Kth smallest element in a BST, checking if a binary tree is symmetric, and level order traversal of a binary tree. However, it does not include a task for finding the height of a binary tree. Therefore, I don't know the answer to the question about finding the height of a binary tree.


In [99]:
result = rag_chain.invoke("Give me DSA questions on Graphs")
result

"The context discusses various topics related to Data Structures and Algorithms (DSA). It includes problems on Mathematics, Bit manipulation, Graphs, and specific algorithms like DFS, BFS, Dijkstra's Algorithm, Kruskal's Algorithm, and methods for detecting cycles and connected components in graphs. However, it does not provide information on what DSA stands for or any general introduction to the field."

# **Happy Coding** !!